In [2]:
from bs4 import BeautifulSoup as bs4
import pandas as pd
import requests
import csv

pages=1
url='https://www.chocolate.co.uk/collections/all'
send_request=requests.get(url)
send_request.status_code
def scrape_all_products(pages=3):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
             }
    
    all_products = []

    for page in range(1, pages + 1):
        url = f"https://www.chocolate.co.uk/collections/all?page={page}"

        try:
            send_request = requests.get(url, headers=headers)
            send_request.raise_for_status()
            soup = bs4(send_request.text, 'html.parser')
        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch page {page}: {e}")
            continue

        
        chocolate_cards = soup.find_all('div', class_='product-item__info')
        print(f"Found {len(chocolate_cards)} products on page {page}")

        for card in chocolate_cards:
           
            title_div = card.find('div', class_='product-item-meta')
            title_tag = title_div.find('a') if title_div else None
            Name = title_tag.text.strip() if title_tag else "No Title"
            product_url = title_tag['href'] if title_tag else None

          
            price_tag = card.find('div', class_='product-item-meta__price-list-container')
            price_span = price_tag.find('span', class_='price') if price_tag else None
            price = price_span.text.strip() if price_span else "No Price"

           
            if product_url:
                product_page = requests.get(f'https://www.chocolate.co.uk{product_url}', headers=headers)
                product_soup = bs4(product_page.text, 'html.parser')
                ingredients = "No Ingredients"
                
                ingredients_div = product_soup.find('div', class_='metafield-rich_text_field')
                if ingredients_div:
                    for p in ingredients_div.find_all('p'):
                        if 'Ingredients:' in p.text:
                            ingredients = p.text.replace("Ingredients:", "").strip()
                            break
            else:
                ingredients = "No URL"
            full_url=f'https://www.chocolate.co.uk{product_url}'
           
            product = {
                "Name": Name,
                "Price": price,
                "Ingredients": ingredients,
                "URL": full_url
            }

            all_products.append(product)

    df = pd.DataFrame(all_products)
    print(f"\nScraped {len(df)} products in total.\n")
    print(df.head())

    return df


data = scrape_all_products(3)

Found 24 products on page 1
Found 24 products on page 2
Found 6 products on page 3

Scraped 54 products in total.

                                      Name                 Price  \
0  2.5kg Bulk 41% Milk Hot Chocolate Drops      Sale price£50.00   
1  2.5kg Bulk 61% Dark Hot Chocolate Drops      Sale price£50.00   
2             41% Milk Hot Chocolate Drops       Sale price£8.75   
3             61% Dark Hot Chocolate Drops       Sale price£8.75   
4                           Almost Perfect  Sale priceFrom £1.00   

                                         Ingredients  \
0  Milk Chocolate (41% Cocoa Minimum, Pure Cocoa ...   
1  Dark Chocolate (61% Cocoa Minimum, Pure Cocoa ...   
2  Milk Chocolate (41% Cocoa Minimum, Pure Cocoa ...   
3  Dark Chocolate (61% Cocoa Minimum, Pure Cocoa ...   
4                                     No Ingredients   

                                                 URL  
0  https://www.chocolate.co.uk/products/2-5kg-bul...  
1  https://www.chocolate.co.u